# Experimental simple performance testing notebook for Pandas
- testing and comparing simple dataframe / sql operations of commong data (pre-)processing tasks 
- various available single-machine Python solutions are to be tested: Pandas, PySpark, Turi Create and Dask.
- execution times, CPU load and maximal memory use should be tracked

## Kiva dataset 
- [Kiva](https://www.kaggle.com/gaborfodor/additional-kiva-snapshot): crowdfunding data with lenders and loans, with additional geographic data
- Pandas Kiva examples: https://www.kaggle.com/gaborfodor/additional-kiva-snapshot/kernels?sortBy=hotness&group=everyone&pageSize=20&datasetId=14345&language=Python


## imports, setup

In [4]:
import timeit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.style.use('fivethirtyeight')
%matplotlib inline

## read files to dataframes: loans and lenders

In [5]:
start = timeit.default_timer()

loans_df = pd.read_csv("../../kiva/loans.csv", parse_dates=['raised_time']) # 2.1 GB, 1.419.607 lines
print('loans: ', loans_df.shape)

lenders_df = pd.read_csv("../../kiva/lenders.csv") #, parse_dates=['raised_time'])  # 130 MB file
print('lenders: ', lenders_df.shape)

print('ellapsed time: ', timeit.default_timer() - start)

loans:  (1419607, 31)
lenders:  (2349174, 11)
ellapsed time:  36.30365106799945


In [10]:
lenders_df.head(2)
loans_df.head(2)

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06,8.0,11,2,1,NaN,female,true,monthly,field_partner


## read, transform and count loan_lenders 
string enumeration to rows: split tuple strings to array, then explode the array to rows


In [2]:
# version 1: this one is the best option
# https://stackoverflow.com/questions/32468402/how-to-explode-a-list-inside-a-dataframe-cell-into-separate-rows
start = timeit.default_timer()

loans_lenders_raw_df = pd.read_csv("kiva/loans_lenders.csv", #nrows=200000, 
                                   dtype={'loan_id': np.int32, 'lenders': object})
# full: 339 MB file, 1.387.433 lines --> 6.3GB, 28.293.931 lines, 388 sec
# 100.000 heading line --> 2.060.259 output lines
# 200.000 heading line --> 4.110.948 output lines, 1.1 GB mem

loans_lenders_raw_df.reset_index(inplace=True)
rows = []
_ = loans_lenders_raw_df.apply(lambda row: [rows.append([row['loan_id'], nn]) 
                         for nn in str.split(str.replace(row.lenders, ' ', ''), ',')], axis=1)

loans_lenders_df = pd.DataFrame(rows, columns=['loan_id', 'lender']) #.set_index(['loan_id', 'lender'])

print('ellapsed time: ', timeit.default_timer() - start)
loans_lenders_df.head(5)

ellapsed time:  374.69706755699985


,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


In [ ]:
# version 2: "The kernel appears to have died. It will restart automatically." after 7 minutes
# https://stackoverflow.com/questions/50729552/split-column-containing-lists-into-different-rows-in-pandas
start = timeit.default_timer()

df = pd.read_csv("kiva/loans_lenders.csv", nrows=200000, dtype={'loan_id': np.int32, 'lenders': object} ) 
# full: 339 MB file, 1.387.433 lines
# 100.000 heading line --> 2.060.259 output lines
# 200.000 heading line --> 

# convert string to list: 
df.loc[:,'lenders'] = df.loc[:,'lenders'].apply(lambda x: str.split(str.replace(x, ' ', ''), ','))

# exploding to rows: 
s = df.apply(lambda x: pd.Series(x['lenders']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'lenders'
df2 = df.drop('lenders', axis=1).join(s)
df2['lenders'] = pd.Series(df2['lenders'], dtype=object)
df2.columns = ['loan_id','lender']

print('ellapsed time: ', timeit.default_timer() - start)
df2

In [ ]:
# version 3: The kernel appears to have died. It will restart automatically. after 5 min.
# also died with 12GB memory
# https://stackoverflow.com/questions/27263805/pandas-when-cell-contents-are-lists-create-a-row-for-each-element-in-the-list
start_time = timeit.default_timer()

df = pd.read_csv("kiva/loans_lenders.csv", nrows=100000) # 339 MB file, 1.387.433 lines
# convert string to list: 
df.loc[:,'lenders'] = df.loc[:,'lenders'].apply(lambda x: str.split(x, ','))
print('df read, arrays constructed.') 

# exploding to rows: 
res = df.set_index(['loan_id'])['lenders'].apply(pd.Series).stack()
res = res.reset_index()
res = res.drop('level_1', axis=1)
res.columns = ['loan_id','lender']

print('ellapsed time: ', timeit.default_timer() - start_time, ' sec.')
res

## join, filter and sort loan and lender data
get distinct joined lines with renamed columns, then write to an output file (for fully materialized results)
- filtering on lenders.country_code: 
  - 'US': 25% of lenders
  - 'CA': 3% of lenders --> 3.5 GB joined

In [4]:
start = timeit.default_timer()

# filter unique lenders: CA: 67.970
lenders_df = lenders_df[lenders_df['country_code']=='CA']

print('ellapsed time: ', timeit.default_timer() - start)

ellapsed time:  0.3428735840000172


In [5]:
start = timeit.default_timer()

# join: 
joined_df_1 = pd.merge(loans_lenders_df, lenders_df, left_on='lender', right_on='permanent_name')
joined_df = pd.merge(joined_df_1, loans_df, on='loan_id')

joined_df.to_csv('kiva/pandas-result-joined.csv') # 3.7 GB

print('ellapsed time: ', timeit.default_timer() - start)

ellapsed time:  179.1593884670001


In [8]:
joined_df.head(5)

,loan_id,lender,permanent_name,display_name,city,state,country_code_x,member_since,occupation,loan_because,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,483693,alan5513,alan5513,Alan,Edmonton,Alberta,CA,1237674751,NaN,A small loan is an investment in enabling some...,...,2012-11-16 18:51:23,8.0,44,2,1,NaN,female,true,irregular,field_partner
1,483693,nick9464,nick9464,nick and anna,victoria,BC,CA,1277253851,self employed,it helps those who need help,...,2012-11-16 18:51:23,8.0,44,2,1,NaN,female,true,irregular,field_partner
2,483693,jason3883,jason3883,Jason C,Toronto,NaN,CA,1317399503,"I ""Like"" It.",for a better tomorrow.,...,2012-11-16 18:51:23,8.0,44,2,1,NaN,female,true,irregular,field_partner
3,483693,maria3124,maria3124,Maria -Ellie,NaN,BC,CA,1258937002,NaN,I' d like to think I can make a diference in s...,...,2012-11-16 18:51:23,8.0,44,2,1,NaN,female,true,irregular,field_partner
4,483693,carolineandcolin9686,carolineandcolin9686,Caroline & Colin,Newington,Ontario,CA,1207318672,retired,We want to share our wealth with others around...,...,2012-11-16 18:51:23,8.0,44,2,1,NaN,female,true,irregular,field_partner


## group and sort joined data
* group by on the joined ‘CA’ table (3.5 GB): count distinct sector_name by lender, then sort

In [30]:
# .nunique(): df.groupby('domain')['ID'].nunique() vs. count(): nunique does not get group names if not in index

start = timeit.default_timer()

country_sum_df = joined_df[['lender', 'sector_name']].groupby(['lender'], as_index=True)['sector_name'].nunique() \
#    .sort_values(by='sector_name', ascending=False)

print('ellapsed time: ', timeit.default_timer() - start)

country_sum_df.head(5)

ellapsed time:  3.6861077610001303


lender
070707Weddingtablegifts    11
07272010                   15
10416122                    6
1096                        5
11lindsaydo                10
Name: sector_name, dtype: int64

In [31]:
country_sum_df.shape  # count: 58.149, nunique: 58149

(58149,)

* group by on the exploded loans_lenders table (6 GB): count distinct loan_id by lender

In [33]:
start = timeit.default_timer()

lender_loan_count_df = loans_lenders_df.groupby(['lender'], as_index=True)['loan_id'].nunique() \
#    .sort_values(by='sector_name', ascending=False)

print('ellapsed time: ', timeit.default_timer() - start)
print(lender_loan_count_df.shape)

lender_loan_count_df.head(5)

ellapsed time:  33.39838374400006
(58149,)


lender
000               40
00000             39
0002              68
00mike00           1
0101craign0101    71
Name: loan_id, dtype: int64

- group by on the loans table (2.1 GB): sum funded_amount by sector_name

In [34]:
start = timeit.default_timer()

loans_sum_df = loans_df[['funded_amount', 'sector_name']].groupby(['sector_name'], as_index=True).sum() \
#    .sort_values(by='sector_name', ascending=False)

print('ellapsed time: ', timeit.default_timer() - start)
print(loans_sum_df.shape)

loans_sum_df.head(5)

ellapsed time:  0.12492709700018167
(15, 1)


,funded_amount
sector_name,
Agriculture,263018585.0
Arts,25730180.0
Clothing,80023415.0
Construction,17463740.0
Education,44216020.0


In [35]:
loans_df.shape

(1419607, 31)